Etape 1:Extraction

In [ ]:
pip install requests

In [12]:
import pandas as pd
import requests

# Définir l'URL de l'API avec les paramètres inclus
url1 = "https://api.apify.com/v2/datasets/7TLO1DTef4Shg4wdM/items?clean=true&format=json&token=apify_api_LNhdNJIYthoRzfbgc6QGfcIAfoTish1Jyn9e"
url2 = "https://api.apify.com/v2/datasets/7TLO1DTef4Shg4wdM/items?clean=true&format=json&view=reviews&token=apify_api_LNhdNJIYthoRzfbgc6QGfcIAfoTish1Jyn9e"

# Envoyer la première requête GET
response1 = requests.get(url1)

# Envoyer la deuxième requête GET
response2 = requests.get(url2)

# Vérifier si les requêtes ont réussi
if response1.status_code == 200 and response2.status_code == 200:
    # Extraire les données JSON de chaque réponse
    data1 = response1.json()
    data2 = response2.json()

    # Créer un DataFrame pandas à partir des données de chaque URL
    df1 = pd.DataFrame(data1)
    df2 = pd.DataFrame(data2)

    # Enregistrer les données de chaque DataFrame dans un fichier Excel
    df1.to_excel("Data1.xlsx", index=False)
    df2.to_excel("Data2.xlsx", index=False)
    print("Données enregistrées avec succès.")
else:
    print("Une des requêtes a échoué.")


Etape 2: Transformation

In [ ]:
# Importer la bibliothèque pandas
import pandas as pd

# Supposons que df1 et df2 sont vos DataFrames
# Chemin d'accès au fichier Excel sur votre bureau
chemin_fichier1 = "Data1.xlsx"
chemin_fichier2 = "Data2.xlsx"

data1 = pd.read_excel(chemin_fichier1)
data2 = pd.read_excel(chemin_fichier2)

# Effectuer la jointure en utilisant la fonction merge()
jointure_df = pd.merge(data1, data2, on='title', how='inner')

# how='inner' signifie que seules les lignes ayant des valeurs correspondantes dans les deux DataFrames seront conservées.
# Vous pouvez également utiliser 'left', 'right' ou 'outer' pour spécifier différents types de jointures.
# Spécifiez le chemin de destination pour le fichier Excel
chemin_destination = "Data.xlsx"

# Enregistrez le DataFrame dans un fichier Excel
jointure_df.to_excel(chemin_destination, index=False)


In [ ]:
#code pour filtrer les colonne
import pandas as pd

# Chemin d'accès au fichier Excel sur votre bureau
chemin_fichier = "Data.xlsx"  # Remplacez "nom_du_fichier.xlsx" par le nom de votre fichier Excel

# Lire le fichier Excel
df = pd.read_excel(chemin_fichier)

# Définir la liste des valeurs de recherche
valeurs_recherche = ['title', 'categoryName', 'address', 'city', 'url', 'reviewsCount', 'reviewsDistribution', 'name', 'text', 'textTranslated', 'publishedAtDate', 'likesCount', 'reviewerNumberOfReviews', 'stars']

# Filtrer les colonnes du DataFrame
df_filtre = df[valeurs_recherche]

# Créer une liste pour stocker les données filtrées
donnees_filtrees = []

# Filtrer les données pour chaque valeur de recherche
for valeur in valeurs_recherche:
    filtered_data = df_filtre[df_filtre.apply(lambda row: valeur in row.astype(str), axis=1)]
    donnees_filtrees.append(filtered_data)

# Concaténer les données filtrées en un seul DataFrame
donnees_filtrees = pd.concat(donnees_filtrees)


# Enregistrer les données filtrées dans un nouveau fichier Excel
donnees_filtrees.to_excel('DataFiltrés.xlsx', index=False)


In [ ]:
from googletrans import Translator
import pandas as pd

# Créer un objet Translator
translator = Translator()

# Fonction pour traduire une chaîne de texte en anglais
def translate_to_english(text):
    try:
        translation = translator.translate(text, src='fr', dest='en')
        return translation.text
    except Exception as e:
        print("Une erreur s'est produite lors de la traduction :", e)
        return ''

# Lire le DataFrame à partir du fichier Excel
df = pd.read_excel('DataFiltrés.xlsx')

# Appliquer la fonction de traduction à la colonne 'textTranslated' et créer une nouvelle colonne 'textTranslated_en'
df['textTranslated_en'] = df['textTranslated'].apply(translate_to_english)

# Afficher le DataFrame avec la nouvelle colonne traduite
df.to_excel('DataWithEnglishTranslation.xlsx', index=False)



In [ ]:
#code pour filtrer les caracteres speciaux
import pandas as pd
import re

# Chemin d'accès au fichier Excel sur votre bureau
chemin_fichier = "DataWithEnglishTranslation.xlsx"  # Remplacez "nom_du_fichier.xlsx" par le nom de votre fichier Excel

# Lire le fichier Excel
df = pd.read_excel(chemin_fichier)

# Prétraitement des commentaires
def preprocess_comment(comment):
    comment = str(comment)  # Convertir en chaîne de caractères si nécessaire
    
    # Supprimer les émojis
    comment = comment.encode('ascii', 'ignore').decode('ascii')
    
    # Convertir en minuscules
    comment = comment.lower()
    
    # Supprimer la ponctuation et les caractères spéciaux
    comment = re.sub(r'[^\w\s]', '', comment)
    
    # Supprimer les espaces supplémentaires
    comment = ' '.join(comment.split())
    
    # Filtrer les commentaires décimaux
    if comment.isdecimal():
        return ''
    
    # Filtrer les commentaires commençant par "CIH"
    if comment.startswith('cih'):
        return ''
    
    return comment

# Appliquer le prétraitement aux commentaires de la colonne "text"
df['textTranslated_en'] = df['textTranslated_en'].apply(preprocess_comment)

# Enregistrer le DataFrame mis à jour dans un nouveau fichier Excel
df.to_excel('DataPreprocessed.xlsx', index=False)


In [ ]:
import pandas as pd

# Charger le DataFrame à partir du fichier Excel
df = pd.read_excel('DataPreprocessed.xlsx')

# Filtrer les lignes ne contenant pas "nope" dans la colonne "textTranslated"
df = df[~df['textTranslated_en'].str.contains('nope')]

# Enregistrer le DataFrame mis à jour dans un nouveau fichier Excel
df.to_excel('DataFiltrésFinal.xlsx', index=False)

print("Données filtrées enregistrées avec succès dans le fichier 'DataFiltrés.xlsx'.")

In [ ]:
import pandas as pd
from transformers import BertTokenizer

# Chemin d'accès au fichier Excel sur votre bureau
chemin_fichier = "DataFiltrésFinal.xlsx"  

# Lire le fichier Excel
df = pd.read_excel(chemin_fichier)

# Charger le tokenizer BERT et télécharger les fichiers du modèle si nécessaire
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizer les commentaires
def tokenize_comment(comment):
    # Appliquer la tokenization avec WordPiece
    tokens = tokenizer.tokenize(comment)
    return tokens

# Appliquer la tokenization aux commentaires de la colonne "textTranslated_en" et créer une nouvelle colonne pour les tokens
df['tokens'] = df['textTranslated_en'].apply(tokenize_comment)

# Enregistrer le DataFrame mis à jour dans un nouveau fichier Excel
df.to_excel('DataTokenized.xlsx', index=False)

In [ ]:
#calcule du score de chaque commentaire
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Charger le fichier Excel
df = pd.read_excel('DataTokenized.xlsx')

# Instancier le tokenizer et le modèle
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Fonction pour calculer le sentiment
def calculate_sentiment(text):
    tokens = tokenizer.encode(text, return_tensors='pt')
    result = model(tokens)
    sentiment = int(torch.argmax(result.logits))+1
    return sentiment

# Appliquer la fonction calculate_sentiment à la colonne 'textTranslated'
df['sentiment'] = df['textTranslated_en'].astype(str).apply(calculate_sentiment)

# Afficher le tableau avec la nouvelle colonne 'sentiment'
print(df)
df.to_excel('DataReviewsTokenized_Sentiment.xlsx', index=False)

In [ ]:
#calcule du totaleScore
import pandas as pd

# Charger le fichier Excel initial
df = pd.read_excel('DataReviewsTokenized_Sentiment.xlsx')


# Calculer le nombre de lignes dans la colonne 'sentiment' groupé par 'address'
result = df.groupby('address')['sentiment'].count()

# Calculer la somme des valeurs dans la colonne 'sentiment' groupée par 'address'
result_sum = df.groupby('address')['sentiment'].sum()

# Calculer le totalScore en divisant la somme par le nombre de lignes pour chaque groupe,
# en arrondissant à un seul chiffre après la virgule
total_score = round(result_sum / result, 1)

# Ajouter la colonne 'totalScore' au DataFrame initial
df['totalScore'] = df['address'].map(total_score)

# Enregistrer les résultats dans un nouveau fichier Excel

df.to_excel('DataTotalScore.xlsx', index=False)